<VSCode.Cell language="markdown">
# HRNet — Cuadernillo completo

Este cuadernillo explica en profundidad HRNet (High-Resolution Network, 2019). Está pensado para tu presentación y estudio: incluye resumen del paper, explicación de la arquitectura paso a paso, implementaciones didácticas en PyTorch, comparaciones con redes tradicionales, ejemplos prácticos (pose estimation y segmentación), visualizaciones y guía para fine-tuning.

Índice dinámico (usar las cabeceras de las celdas para navegar):

1. Import Required Libraries
2. HRNet Architecture Overview
3. High-Resolution Representation Maintenance (demo)
4. Multi-Resolution Parallel Convolutions
5. Multi-Scale Fusion Modules
6. HRNet Implementation from Scratch
7. Comparison with Traditional CNN Architectures
8. HRNet for Human Pose Estimation
9. HRNet for Semantic Segmentation
10. Performance Benchmarking
11. Visualization of Feature Maps
12. Transfer Learning with Pre-trained HRNet

---

> Nota: todo el código es ejecutable en CPU; si tienes GPU activada, PyTorch la usará automáticamente.
</VSCode.Cell>

In [ ]:
<VSCode.Cell language="python">
# 1. Import Required Libraries
import sys
import math
import time
import random
from dataclasses import dataclass

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

print('PyTorch version:', torch.__version__)
print('Device available:', 'cuda' if torch.cuda.is_available() else 'cpu')
</VSCode.Cell>

<VSCode.Cell language="markdown">
## HRNet Architecture Overview

Resumen del paper original (Sun et al., 2019 - "Deep High-Resolution Representation Learning for Visual Recognition").

- Idea principal: mantener representaciones de alta resolución durante todo el procesamiento. En vez de aplicar una cadena de convoluciones que bajan progresivamente la resolución (como ResNet), HRNet mantiene varias ramas en paralelo a diferentes resoluciones y realiza fusiones repetidas entre ellas.
- Dos características clave:
  1. Conexiones en paralelo entre flujos de alta a baja resolución.
  2. Intercambio repetido de información entre resoluciones (multi-scale fusion).

Ventajas: representaciones espaciales más precisas, mejores predicciones para tareas sensibles a la posición (pose estimation, segmentación, face alignment).

Referencias: https://arxiv.org/abs/1908.07919
</VSCode.Cell>

In [ ]:
<VSCode.Cell language="python">
# Diagrama simplificado de HRNet (usando matplotlib)
plt.figure(figsize=(6,3))
plt.text(0.05,0.8,'Entrada\n(High-res)', fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.text(0.05,0.5,'Stem\nConv', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
plt.arrow(0.25,0.7,0.2,0, head_width=0.02)
plt.text(0.5,0.8,'Rama alta\n(res 1x)', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
plt.text(0.5,0.6,'Rama baja\n(res 1/2)', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
plt.text(0.5,0.4,'Rama más baja\n(res 1/4)', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
plt.arrow(0.75,0.7,0.15,0, head_width=0.02)
plt.text(0.93,0.7,'Fusión\n(repeated)', fontsize=10)
plt.axis('off')
plt.title('Esquema simplificado: HRNet mantiene ramas en paralelo y realiza fusiones repetidas')
plt.show()

# Demo rápido: comparar shapes de un flujo ResNet-like y HRNet-like con tensor aleatorio
x = torch.randn(1,3,256,256)
# ResNet-like: reduce resolución por 2 dos veces
resnet_shapes = []
h = x
for i in range(3):
    resnet_shapes.append(h.shape)
    h = F.avg_pool2d(h, kernel_size=2)
resnet_shapes.append(h.shape)

# HRNet-like: mantener tres ramas a distintas resoluciones sin reducir la rama alta
hr_shapes = [x.shape, (1,64,128,128), (1,128,64,64)]

print('ResNet-like shapes (downsampling chain):')
for s in resnet_shapes:
    print(' ', s)
print('\nHRNet-like (parallel branches examples):')
for s in hr_shapes:
    print(' ', s)
</VSCode.Cell>

In [ ]:
<VSCode.Cell language="python">
# 4. Multi-Resolution Parallel Convolutions
class ParallelBranch(nn.Module):
    """Una rama simple que procesa una resolución dada."""
    def __init__(self, in_channels, out_channels, blocks=2):
        super().__init__()
        layers = []
        for _ in range(blocks):
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
            in_channels = out_channels
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# 5. Multi-Scale Fusion Modules
class FuseLayer(nn.Module):
    """Fusiona una lista de características con diferentes resoluciones.
    Para subir/ bajar resolución utiliza conv 1x1 + upsample/avg_pool.
    """
    def __init__(self, in_channels_list, out_channels_list):
        super().__init__()
        self.num_branches = len(in_channels_list)
        # para cada par (i->j) formamos una transformación
        self.transforms = nn.ModuleList()
        for i in range(self.num_branches):
            row = nn.ModuleList()
            for j in range(self.num_branches):
                if i == j:
                    row.append(nn.Identity())
                elif i < j:
                    # i es más alta resolución que j: downsample (avg pool)
                    ops = nn.Sequential(
                        nn.Conv2d(in_channels_list[i], out_channels_list[j], kernel_size=1, bias=False),
                        nn.BatchNorm2d(out_channels_list[j])
                    )
                    row.append(ops)
                else:
                    # i > j: upsample
                    ops = nn.Sequential(
                        nn.Conv2d(in_channels_list[i], out_channels_list[j], kernel_size=1, bias=False),
                        nn.BatchNorm2d(out_channels_list[j])
                    )
                    row.append(ops)
            self.transforms.append(row)

    def forward(self, x_list):
        # x_list: lista de tensores [B,C,H,W] de distintas resoluciones (res0: alta)
        out = []
        for j in range(self.num_branches):
            y = 0
            for i in range(self.num_branches):
                xi = x_list[i]
                transf = self.transforms[i][j]
                if i == j:
                    contrib = transf(xi)
                elif i < j:
                    # bajar resolución: aplicar transf y luego pool
                    contrib = transf(xi)
                    factor = 2 ** (j - i)
                    contrib = F.avg_pool2d(contrib, kernel_size=factor)
                else:
                    # subir resolución: aplicar transf y luego upsample
                    contrib = transf(xi)
                    scale = 2 ** (i - j)
                    contrib = F.interpolate(contrib, scale_factor=scale, mode='nearest')
                y = y + contrib
            out.append(F.relu(y))
        return out

# Prueba rápida de los módulos
a = torch.randn(1,32,128,128)
b = torch.randn(1,64,64,64)
c = torch.randn(1,128,32,32)
branches = [ParallelBranch(3,32), ParallelBranch(32,64), ParallelBranch(64,128)]
# solo para mostrar forward con las formas de ejemplo
x_list = [torch.randn(1,32,128,128), torch.randn(1,64,64,64), torch.randn(1,128,32,32)]
fl = FuseLayer([32,64,128],[32,64,128])
out = fl(x_list)
for o in out:
    print('->', o.shape)
</VSCode.Cell>

In [ ]:
<VSCode.Cell language="python">
# 6. HRNet Implementation from Scratch (simplificado)
class SimpleHRNet(nn.Module):
    def __init__(self, num_classes=1000, widths=(32,64,128)):
        super().__init__()
        # Stem: conv inicial
        self.stem = nn.Sequential(
            nn.Conv2d(3, widths[0], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(widths[0]),
            nn.ReLU(inplace=True)
        )
        # Branches: una por cada resolución
        self.branches = nn.ModuleList()
        for i,w in enumerate(widths):
            in_ch = widths[0] if i==0 else widths[i]
            self.branches.append(ParallelBranch(in_ch, w, blocks=2))
        # Fusions repetidas (dos repeticiones para ejemplificar)
        self.fuse1 = FuseLayer(widths, widths)
        self.fuse2 = FuseLayer(widths, widths)
        # Head
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(widths[0], num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        # crear entradas para cada rama: la rama 0 es la salida del stem
        x_list = [x, F.avg_pool2d(x,2), F.avg_pool2d(F.avg_pool2d(x,2),2)]
        # pasar por ramas
        x_list = [self.branches[i](x_list[i]) for i in range(len(self.branches))]
        # fusión repetida
        x_list = self.fuse1(x_list)
        x_list = self.fuse2(x_list)
        # clasificador tomando la rama de mayor resolución
        out = self.head(x_list[0])
        return out

# prueba rápida
model = SimpleHRNet(num_classes=10)
input_tensor = torch.randn(2,3,256,256)
out = model(input_tensor)
print('Output shape:', out.shape)

# mostrar número aproximado de parámetros
n_params = sum(p.numel() for p in model.parameters())
print('Params (approx):', n_params)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 7. Comparison with Traditional CNN Architectures

Breve comparación conceptual:

- ResNet: flujo en serie, baja resolución acumulativa y luego upsampling si es necesario. Bueno para clasificación.
- U-Net: encoder-decoder con skip connections; mantiene detalles gracias a skip pero primero reduce resolución.
- HRNet: mantiene alta resolución durante todo el camino y agrega ramas de baja resolución en paralelo; intercambia información frecuentemente.

Ventaja práctica: mejores predicciones en tareas sensibles a la localización espacial.
</VSCode.Cell>

In [ ]:
<VSCode.Cell language="python">
# 8. HRNet for Human Pose Estimation (simplificado)
class KeypointHead(nn.Module):
    def __init__(self, in_channels, num_joints=17):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(in_channels)
        self.out = nn.Conv2d(in_channels, num_joints, kernel_size=1)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        return self.out(x)

# Prueba con la rama de mayor resolución del HRNet simple
model = SimpleHRNet(num_classes=10)
with torch.no_grad():
    x = torch.randn(1,3,256,256)
    # simular obtener la rama de resolución alta desde el forward (rearange)
    stem = model.stem(x)
    high_res = stem
    khead = KeypointHead(high_res.shape[1], num_joints=17)
    heatmaps = khead(high_res)
    print('Heatmaps shape (B, joints, H, W):', heatmaps.shape)

# 9. HRNet for Semantic Segmentation (simplificado)
class SegmentationHead(nn.Module):
    def __init__(self, in_channels, n_classes=21):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels//2, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels//2)
        self.conv2 = nn.Conv2d(in_channels//2, n_classes, kernel_size=1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.interpolate(self.conv2(x), scale_factor=1, mode='bilinear', align_corners=False)
        return x

seg_head = SegmentationHead(high_res.shape[1], n_classes=21)
seg_logits = seg_head(high_res)
print('Seg logits shape:', seg_logits.shape)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 10. Performance Benchmarking

La siguiente celda mide tiempos de forward pase en CPU y en GPU si está disponible. También recoge memoria GPU si es posible.

## 11. Visualization of Feature Maps

Mostraremos mapas de características intermedios de la rama de alta resolución.

## 12. Transfer Learning with Pre-trained HRNet

Explicación rápida de cómo cargar pesos desde el model zoo oficial y adaptar la cabeza. El código intentará descargar los pesos, pero en caso de no poder hacerlo, dará instrucciones para que los descargues manualmente.

---

### Try it

Comandos para crear entorno y lanzar el notebook (PowerShell):

```powershell
python -m venv .venv; .\.venv\Scripts\Activate.ps1
pip install --upgrade pip
pip install torch torchvision matplotlib seaborn pillow
jupyter notebook "HRNet_Cuadernillo.ipynb"
```
</VSCode.Cell>